# Writing checks

Checks intention is to give students a way to validate their code solution. The student's code can be validated by providing a list of inputs and references comparing the output of the student's code with the references, or by directy testing certain functional behavior of the code. In cases when reference outputs need to be obfuscated the outputs that are compared can be passed through a _fingerprint_ function. This notebook goes through each of these features and presents an example.

In [ ]:
from scwidgets import CodeInput, CodeExercise, Check, CheckRegistry

import numpy as np 

Similar as for the `ExerciseRegistry` we need to define a `CheckRegistry` that registers the checks for each exercise.

In [ ]:
check_registry = CheckRegistry()

## Checks using inputs and output references

In [ ]:
def sin(arr):
    import numpy as np
    return np.cos(arr) # oops! wrong solution


check_code_ex = CodeExercise(
    exercise_key="sinus_with_references_1",
    code=sin,
    check_registry=check_registry,
)

# An assert function returns a string that specifies
# the error message to the student, or is empty if the
# check passes
def my_assert_allclose(outputs, references) -> str:
    if not np.allclose(outputs, references):
        return "Your output is not close to the references."
    else: 
        return "" # We use empty strings means it passes

check_registry.add_check(
    check_code_ex,
    asserts=[
        my_assert_allclose
    ],
    inputs_parameters=[{"arr": np.asarray([0., np.pi, 2*np.pi])}],
    outputs_references=[(np.asarray([0., 0., 0.]),)]
)

check_code_ex.run_check()
check_code_ex

Because there are asserts that are repeatedly needed for almost any kind of exercise, we provide of a set of asserts

In [ ]:
from scwidgets import (
    assert_numpy_allclose,
    assert_shape,
    assert_type,
)

def sinus(arr):
    import numpy as np
    return np.cos(arr) # oops! wrong solution

code_input_sinus = CodeInput(sinus)

check_code_ex = CodeExercise(
    exercise_key="sinus_with_references_2",
    code=code_input_sinus,
    check_registry=check_registry,
)

check_registry.add_check(
    check_code_ex,
    asserts=[
        assert_type, # checks if same type as reference values 
        assert_shape, # checks if same shape as reference values
        assert_numpy_allclose, # checks if allclose to reference values
    ],
    inputs_parameters=[{"arr": np.asarray([0., 0.78539816, 1.57079633, 2.35619449, 3.14159265])}],
    outputs_references=[(np.asarray([0., 7.07106781e-01, 1.00000000e+00, 7.07106781e-01, 0.]),)]
)

check_code_ex.run_check()
check_code_ex

In [ ]:
print(code_input_sinus.full_function_code)

One can adapt the default arguments by using partial functions

In [ ]:
assert_numpy_allclose?

In [ ]:
from functools import partial

custom_assert_numpy_allclose = partial(assert_numpy_allclose, rtol=1e-7)

## Testing functional behavior

In [ ]:
def sinus(arr):
    import numpy as np
    return np.cos(arr) # oops! wrong solution

code_input_sinus = CodeInput(sinus)

check_code_ex = CodeExercise(
    exercise_key="sinus_functional_behavior",
    code=code_input_sinus,
    check_registry=check_registry,
)

def assert_2pi_periodic() -> str:
    out = code_input_sinus.get_function_object()([0, 2*np.pi])
    if not np.allclose(out[0], out[1]):
        return "Function is not periodic."
    return "" # empty strings means it passes

check_registry.add_check(
    check_code_ex,
    asserts=[
        assert_2pi_periodic,
    ]
)

check_code_ex.run_check()
check_code_ex

## Obfuscating the reference solution with fingerprint

In [ ]:
from scwidgets.check import (
    assert_equal
)

def riddle():
    """
    Please return as string the answer to this riddle:
    
    What has wings but in the air it not swings.
    I looked to the north, but it was not worth.
    What I am looking for?
    """
    return ""
code_input_sinus = CodeInput(riddle)

check_code_ex = CodeExercise(
    exercise_key="riddle",
    code=code_input_sinus,
    check_registry=check_registry,
)

#def assert_equal(output, reference):
#    return "" if output == reference else "Not correct solution. Hint: it is an animal in the antarctica."

char_to_num = {char: num for num, char in enumerate("abcdefghijklmnopqrmnstuvwxyz")}
def string_to_int(output):
    return sum([char_to_num[char] for char in output])

check_registry.add_check(
    check_code_ex,
    asserts=[
        assert_equal,
    ],
    fingerprint = string_to_int,
    inputs_parameters=[{}],
    outputs_references=[(93,),],
    suppress_fingerprint_asserts = True # By default we do not show the error message, since it is confusing with the fingerprint
)

check_code_ex.run_check()
check_code_ex

### Solution

In [ ]:
# Once you enter the solution you can get the reference output with
string_to_int(check_code_ex.compute_output_to_check())

## Checking all widgets

The check registry also provides the possibility to check all the widgets. 

In [ ]:
check_registry

We create a widget that will raise an error to show how this is visualized.

In [ ]:
def error(arr):
    raise ValueError("Oops!")
    return arr

check_code_ex = CodeExercise(
    exercise_key="will_raise_error",
    code=error,
    check_registry=check_registry,
)

check_registry.add_check(
    check_code_ex,
    asserts=[
        assert_type,
    ],
    inputs_parameters=[{"arr": np.asarray([1., 2., 3.5])}],
    outputs_references=[(np.asarray([1., 2., 3.5]),)]
)

check_code_ex.run_check()
check_code_ex

In [ ]:
# For the demo to automatically run we simulate a button press using the private function that should not be used
check_registry._check_all_widgets_button.click()